In [13]:
# Setup
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from math import sqrt
from sklearn.preprocessing import MinMaxScaler

import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [14]:
# supress warnings
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Load Dataset
df = pd.read_csv('/content/drive/MyDrive/Bangkit/Dataset/newAllDataset.csv', header=0, index_col=0, parse_dates=True)
df

,Bayam,Kangkung,Sawi,Brokoli,Kubis,Wortel,Kentang,Kacang Polong,Kacang Tanah,Buncis,...,Lada,Cengkeh,Kayu Manis,Kapulaga,Kunyit,Ubi Jalar,Ubi Kayu,Kentang.1,Singkong,Talas
Tanggal,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1500,1000,5500,9000,3500,8000,6000,30000,26000,17000,...,89000,130000,17000,83000,3500,3000,4000,2000,1000,14000
2013-08-01,1500,1000,5500,9000,3500,8000,6000,30000,26000,17000,...,89000,130000,17000,83000,3500,3000,4000,2000,1000,14000
2013-01-15,1500,1000,5500,9000,3500,8000,6000,30000,26000,16000,...,89000,130000,17000,83000,3500,3000,4000,2000,1000,14000
2013-01-22,1500,1000,5500,9000,3500,8000,6000,30000,26000,16000,...,89000,130000,17000,83000,3500,3000,4000,2000,1000,14000
2013-01-29,1500,1000,5500,9000,3500,8000,6000,30000,26000,16000,...,89000,130000,17000,83000,3500,3000,4000,2000,1000,14000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,11500,11000,7000,38000,13500,13000,18000,57000,27500,35000,...,125500,180000,23000,82000,9500,9000,5500,5000,2000,22000
2023-07-07,12000,11500,6000,38000,15000,18000,18000,57000,28000,35000,...,125500,180000,23000,82000,9500,9000,5500,5000,2000,22000
2023-07-14,12000,11500,6000,38000,15000,18000,18000,57000,28000,35000,...,125500,180000,23000,82000,9500,9000,5500,5000,2000,22000


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 553 entries, 2013-01-01 to 2023-07-28
Data columns (total 54 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Bayam                 553 non-null    int64  
 1   Kangkung              553 non-null    int64  
 2   Sawi                  553 non-null    int64  
 3   Brokoli               553 non-null    int64  
 4   Kubis                 553 non-null    int64  
 5   Wortel                553 non-null    int64  
 6   Kentang               553 non-null    int64  
 7   Kacang Polong         553 non-null    int64  
 8   Kacang Tanah          553 non-null    int64  
 9   Buncis                553 non-null    int64  
 10  Labu Siam             553 non-null    int64  
 11  Labu Kuning           553 non-null    int64  
 12  Lobak Merah           553 non-null    int64  
 13  Lobak Putih           553 non-null    int64  
 14  Tomat                 553 non-null    int64  
 15  Tero

In [20]:
# Fungsi untuk Membentuk Data untuk Model
def df_to_X_y(df, window_size=75):
      df_as_np = df.to_numpy()
      X = []
      y = []

      for i in range(len(df_as_np)-window_size):
          row = [[a] for a in df_as_np[i:i+window_size]]
          X.append(row)
          label = df_as_np[i+window_size]
          y.append(label)
      return np.array(X), np.array(y)

In [23]:
class MyCallback(tf.keras.callbacks.Callback):
    def __init__(self, target_rmse=0.15):
        super(MyCallback, self).__init__()
        self.target_rmse = target_rmse

    def on_epoch_end(self, epoch, logs=None):
        val_rmse = logs.get('val_root_mean_squared_error')
        if val_rmse is not None and val_rmse < self.target_rmse:
            print(f"\nReached target RMSE ({self.target_rmse}), stopping training.")
            self.model.stop_training = True

In [27]:
jumlah_kolom = df.shape[1]
jumlah_kolom

for i in range(jumlah_kolom):
  nama_produk = df.columns[i]
  series_harga = df[nama_produk]

  # Normalisasi Data
  mean_value = series_harga.mean()
  std_value = series_harga.std()
  normalized_series = (series_harga - mean_value) / std_value

  WINDOW_SIZE = 5
  X, y = df_to_X_y(normalized_series, WINDOW_SIZE)
  X.shape, y.shape

  # Pembagian Data menjadi Train, Validation, dan Test sets
  X_train, y_train = X[:500], y[:500]
  X_val, y_val = X[500:520], y[500:520]
  X_test, y_test = X[520:], y[520:]

  # Inisialisasi scaler
  scaler = MinMaxScaler(feature_range=(0, 1))

  # Normalisasi X_train
  X_train = scaler.fit_transform(X_train.reshape(-1, WINDOW_SIZE))

  # Normalisasi X_val
  X_val = scaler.transform(X_val.reshape(-1, WINDOW_SIZE))

  # Normalisasi X_test
  X_test = scaler.transform(X_test.reshape(-1, WINDOW_SIZE))

  model = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape=(WINDOW_SIZE, 1)),
      tf.keras.layers.LSTM(64, return_sequences='True'),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(8, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])

  # Kompilasi dan Pelatihan Model dengan Normalisasi
  # checkpoint = tf.keras.callbacks.ModelCheckpoint('model/', save_best_only=True)
  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  callbacks = MyCallback(target_rmse=0.01)
  model.fit(X_train,
            y_train,
            validation_data=(X_val, y_val),
            epochs=1000,
            callbacks=[callbacks])

  train_predict=model.predict(X_train)
  test_predict=model.predict(X_test)

  print("\n\nTrain data RMSE ", nama_produk," : ", math.sqrt(mean_squared_error(y_train,train_predict)))
  print("Train data MSE " , nama_produk," : ", mean_squared_error(y_train,train_predict))
  print("Train data MAE " , nama_produk," : ", mean_absolute_error(y_train,train_predict))
  print("-------------------------------------------------------------------------------------")
  print("Test data RMSE " , nama_produk," : ", math.sqrt(mean_squared_error(y_test,test_predict)))
  print("Test data MSE "  , nama_produk," : ", mean_squared_error(y_test,test_predict))
  print("Test data MAE "  , nama_produk," : ", mean_absolute_error(y_test,test_predict))

  # Save your model in the SavedModel format
  model.save(f"model_{nama_produk}.h5")

Epoch 1/10
16/16 [==============================] - 5s 72ms/step - loss: 0.9685 - root_mean_squared_error: 0.9841 - val_loss: 0.3932 - val_root_mean_squared_error: 0.6270
Epoch 2/10
16/16 [==============================] - 0s 9ms/step - loss: 0.9481 - root_mean_squared_error: 0.9737 - val_loss: 0.3706 - val_root_mean_squared_error: 0.6088
Epoch 3/10
16/16 [==============================] - 0s 10ms/step - loss: 0.9284 - root_mean_squared_error: 0.9635 - val_loss: 0.3423 - val_root_mean_squared_error: 0.5851
Epoch 4/10
16/16 [==============================] - 0s 9ms/step - loss: 0.9017 - root_mean_squared_error: 0.9496 - val_loss: 0.2971 - val_root_mean_squared_error: 0.5451
Epoch 5/10
16/16 [==============================] - 0s 9ms/step - loss: 0.8682 - root_mean_squared_error: 0.9318 - val_loss: 0.2479 - val_root_mean_squared_error: 0.4979
Epoch 6/10
16/16 [==============================] - 0s 11ms/step - loss: 0.8315 - root_mean_squared_error: 0.9118 - val_loss: 0.2040 - val_root_mean